In [1]:
from scipy.io import arff
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import keras.backend as K
from keras import regularizers
import keras_tuner as kt

#macro post evaluation Recall, Precision, F1 & Acc each Label
def recall_macro(y_true,y_pred):
    true_positives = np.sum(y_true * y_pred, axis=0)
    possible_positives = np.sum(y_true, axis=0)
    recall_each_label = true_positives / possible_positives
    recall_macro = np.mean(recall_each_label)
    return recall_each_label, recall_macro

def precision_macro(y_true,y_pred):
    true_positives = np.sum(y_true * y_pred, axis=0)
    predicted_positives = possible_positives = np.sum(y_pred, axis=0)
    precision_each_label = true_positives / predicted_positives
    precision_macro = np.mean(precision_each_label)
    return precision_each_label, precision_macro
def f1_macro(y_true,y_pred):
    p = precision_macro(y_true,y_pred)[0]
    r = recall_macro(y_true,y_pred)[0]
    f1_each_label = 2*((p*r)/(p+r))
    f1_macro = np.mean(f1_each_label)    
    return f1_each_label,f1_macro
def accuracy_each_label(y_true,y_pred):
    acc_prediction = np.count_nonzero(y_true - y_pred==0,axis=0)
    acc = acc_prediction / np.shape(y_true)[0]
    return acc
    
#micro integrated in Model Recall, Precision, F1
def recall_micro(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall_micro = true_positives / (possible_positives + K.epsilon())
    return recall_micro
def precision_micro(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision_micro = true_positives / (predicted_positives + K.epsilon())
    return precision_micro
def f1_micro(y_true, y_pred):
    p = precision_micro(y_true, y_pred)
    r = recall_micro(y_true, y_pred)
    f1_micro= 2*((p*r)/(p+r+K.epsilon()))
    return f1_micro

#hamming integrated in model
def hamming_loss(y_true, y_pred):
    temp = K.abs(y_true-y_pred)
    return K.mean(K.cast(K.greater(temp,0.5),dtype=float))